# Data 

In [56]:
import torch
torch.set_default_dtype(torch.float64)

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [58]:
import os
!wget https://raw.githubusercontent.com/quaternion12345/lora/main/20210416.csv
os.rename(r'20210416.csv',r'rssi_data.csv')

--2021-05-07 08:59:58--  https://raw.githubusercontent.com/quaternion12345/lora/main/20210416.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3478 (3.4K) [text/plain]
Saving to: ‘20210416.csv’

20210416.csv        100%[===================>]   3.40K  --.-KB/s    in 0s      

2021-05-07 08:59:58 (58.3 MB/s) - ‘20210416.csv’ saved [3478/3478]



In [59]:
rssi_data_df = pd.read_csv("rssi_data.csv",index_col=0)
rssi_data_df.head(n=10)

,rssi.202,rssi.245,rssi.129,rssi.122,outcome
1,-84,-96,-55,-71,1
2,-96,-87,-54,-70,1
3,-73,-81,-64,-71,1
4,-81,-78,-60,-73,1
5,-75,-79,-65,-73,1
6,-79,-88,-62,-77,1
7,-75,-80,-66,-68,1
8,-94,-79,-60,-65,1
9,-74,-79,-63,-71,1
10,-76,-86,-74,-68,1


In [60]:
max(rssi_data_df["rssi.202"])

-55

In [61]:
np.mean(rssi_data_df["rssi.202"])

-72.63398692810458

In [62]:
np.std(rssi_data_df["rssi.202"])

5.875340773303582

In [63]:
rssi_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 1 to 153
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   rssi.202  153 non-null    int64
 1   rssi.245  153 non-null    int64
 2   rssi.129  153 non-null    int64
 3   rssi.122  153 non-null    int64
 4   outcome   153 non-null    int64
dtypes: int64(5)
memory usage: 7.2 KB


In [70]:
train_dataset[0]

{'features': array([74, 81, 77, 73]), 'outcomes': array(3)}

# Dense model

In [117]:
class DenseModel(nn.Module):
    def __init__(self, num_in):

        super(DenseModel, self).__init__()

        self.fc1 = nn.Linear(num_in, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.ouput = nn.Linear(64, 11)
        #self.dropout = F.dropout(p=0.5)
        #self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = F.gelu(self.fc2(x))
        x = F.gelu(self.fc3(x))
        x = F.gelu(self.fc4(x))

        #x = F.dropout(torch.sigmoid(self.fc3(x)), p=0.5)
        x = self.ouput(x)
        return x

dense = DenseModel(4)


In [150]:
from sklearn.model_selection import KFold
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

In [160]:
# Configuration options
k_folds = 5
num_epochs = 250
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

kfold = KFold(n_splits=k_folds, shuffle=True)
dataset = rssi_data_df.values # TensorFlow/Keras likes numpy/arrays rather than Dataframes

In [161]:
# K-fold Cross Validation model evaluation
network = DenseModel(4)

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
  
  # Print
  print(f'FOLD {fold}')
  print('--------------------------------')
  
  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  # Define data loaders for training and testing data in this fold
  trainloader = torch.utils.data.DataLoader(
                    dataset, 
                    batch_size=10, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=10, sampler=test_subsampler)
  
  # Init the neural network
  # network = DenseModel(4)
  # network.apply(reset_weights)

  
  # Initialize optimizer
  optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
  
  # Run the training loop for defined number of epochs
  for epoch in range(0, num_epochs):

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):

      # Get inputs
      inputs = data[:,:4]
      targets = data[:,4]
      # Zero the gradients
      optimizer.zero_grad()
      
      # Perform forward pass
      outputs = network(inputs.double())
      
      # Compute loss
      loss = loss_function(outputs, targets)
      
      # Perform backward pass
      loss.backward()
      
      # Perform optimization
      optimizer.step()
      
      # Print statistics
      current_loss += loss.item()
      if i % 10 == 0:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0
          
  # Process is complete.
  print('Training process has finished. Saving trained model.')

  # Print about testing
  print('Starting testing')

  # Saving the model
  save_path = f'./model-fold-{fold}.pth'
  torch.save(network.state_dict(), save_path)

  # Evaluation for this fold
  correct, total = 0, 0
  with torch.no_grad():

    # Iterate over the test data and generate predictions
    for i, data in enumerate(testloader, 0):

      # Get inputs
      inputs = data[:,:4]
      targets = data[:,4]

      # Generate outputs
      outputs = network(inputs.double())

      # Set total and correct
      _, predicted = torch.max(outputs.data, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()

    # Print accuracy
    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
    print('--------------------------------')
    results[fold] = 100.0 * (correct / total)

FOLD 0
--------------------------------
Starting epoch 1
Loss after mini-batch     1: 0.006
Loss after mini-batch    11: 0.055
Starting epoch 2
Loss after mini-batch     1: 0.004
Loss after mini-batch    11: 0.050
Starting epoch 3
Loss after mini-batch     1: 0.005
Loss after mini-batch    11: 0.046
Starting epoch 4
Loss after mini-batch     1: 0.004
Loss after mini-batch    11: 0.042
Starting epoch 5
Loss after mini-batch     1: 0.004
Loss after mini-batch    11: 0.039
Starting epoch 6
Loss after mini-batch     1: 0.004
Loss after mini-batch    11: 0.039
Starting epoch 7
Loss after mini-batch     1: 0.005
Loss after mini-batch    11: 0.039
Starting epoch 8
Loss after mini-batch     1: 0.004
Loss after mini-batch    11: 0.035
Starting epoch 9
Loss after mini-batch     1: 0.003
Loss after mini-batch    11: 0.034
Starting epoch 10
Loss after mini-batch     1: 0.003
Loss after mini-batch    11: 0.034
Starting epoch 11
Loss after mini-batch     1: 0.003
Loss after mini-batch    11: 0.032
S

In [162]:
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} %')
  sum += value
print(f'Average: {sum/len(results.items())} %')

K-FOLD CROSS VALIDATION RESULTS FOR 4 FOLDS
--------------------------------
Fold 0: 43.58974358974359 %
Fold 1: 47.368421052631575 %
Fold 2: 50.0 %
Fold 3: 55.26315789473685 %
Average: 49.055330634278 %
